In [1]:
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from time import sleep
from bs4 import BeautifulSoup
import re
from ipywidgets import IntProgress
from IPython.display import display

In [2]:
# Spotify API Keys
spotify_cliend_id='bca78196e824433fbdf88ec18d84825f'
spotify_client_secret='d43763215bd8435eb9b3faaf048ca038'
os.environ['SPOTIPY_CLIENT_ID']=spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=spotify_client_secret

In [3]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [27]:
def get_features(title, artist):
    artist = str(artist)
    if ',' in artist:
        artists = artist.split(',')
        for artist in artists:
            features = get_features(title, artist)
            if features is not None:
                return features
            
    results = sp.search(q=f'track:{title} artist:{artist}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'{title}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        return
    track_info = results['tracks']['items'][0]
    track_id = track_info['id']
    features = sp.audio_features([track_id])[0]
    
    if features is None:
        return
    
    features_as_dict = {
        'URL': track_info['external_urls']['spotify'],
        'TrackId': track_id,
        'Acousticness': features['acousticness'],
        'Danceability': features['danceability'],
        'Duration (ms)': features['duration_ms'],
        'Energy': features['energy'],
        'Instrumentalness': features['instrumentalness'],
        'Key': features['key'],
        'Liveness': features['liveness'],
        'Loudness': features['loudness'],
        'Mode': features['mode'],
        'Speechiness': features['speechiness'],
        'Tempo': features['tempo'],
        'Time Signature': features['time_signature'],
        'Valence': features['valence'],
        'Album': track_info['album']['name'],
        'Image': track_info['album']['images'][0]['url'] if len(track_info['album']['images'])>0 else None,
        'Explicit': track_info['explicit'],
        'Popularity': track_info['popularity'],
        'Preview': track_info['preview_url']
    }
    return features_as_dict

In [5]:
get_features('Roadhouse Blues', 'Doors')

{'URL': 'https://open.spotify.com/track/1Q5kgpp4pmyGqPwNBzkSrw',
 'TrackId': '1Q5kgpp4pmyGqPwNBzkSrw',
 'Acousticness': 0.337,
 'Danceability': 0.612,
 'Duration (ms)': 243827,
 'Energy': 0.634,
 'Instrumentalness': 4.6e-05,
 'Key': 9,
 'Liveness': 0.147,
 'Loudness': -8.212,
 'Mode': 1,
 'Speechiness': 0.0319,
 'Tempo': 121.059,
 'Time Signature': 4,
 'Valence': 0.904,
 'Album': 'Morrison Hotel',
 'Image': 'https://i.scdn.co/image/34b1c4afdd8e576fb048e9e6c900c6c9fe33ea76',
 'Explicit': False,
 'Popularity': 71,
 'Preview': 'https://p.scdn.co/mp3-preview/ce4e9952e1519b9fe6c858b085fbe79077ec9dfb?cid=bca78196e824433fbdf88ec18d84825f'}

In [6]:
get_features('Mame', 'Angela Lansbury, Bea Arthur, Jane Connell, Charles Braswell, Jerry Lanning, Frankie Michaels')

{'URL': 'https://open.spotify.com/track/4nmD7mHFACe4XwcoZ7wWsy',
 'TrackId': '4nmD7mHFACe4XwcoZ7wWsy',
 'Acousticness': 0.536,
 'Danceability': 0.355,
 'Duration (ms)': 344853,
 'Energy': 0.828,
 'Instrumentalness': 5.6e-05,
 'Key': 2,
 'Liveness': 0.931,
 'Loudness': -10.261,
 'Mode': 0,
 'Speechiness': 0.487,
 'Tempo': 136.013,
 'Time Signature': 4,
 'Valence': 0.238,
 'Album': 'Tap Your Troubles Away! - The Words and Music of Jerry Herman',
 'Image': 'https://i.scdn.co/image/4b4b77ce331bbbbc7d696516649ad758d632f313',
 'Explicit': False,
 'Popularity': 3,
 'Preview': 'https://p.scdn.co/mp3-preview/ca80ddfa159bc9ec96b615e6c106d1e2efb8978f?cid=bca78196e824433fbdf88ec18d84825f'}

In [28]:
get_features('Ella Swings Lightly', 'Ella Fitzgerald')

{'URL': 'https://open.spotify.com/track/3ZxPZcLbINzbetZFcJthvU',
 'TrackId': '3ZxPZcLbINzbetZFcJthvU',
 'Acousticness': 0.838,
 'Danceability': 0.627,
 'Duration (ms)': 156467,
 'Energy': 0.131,
 'Instrumentalness': 0,
 'Key': 10,
 'Liveness': 0.223,
 'Loudness': -13.779,
 'Mode': 1,
 'Speechiness': 0.0503,
 'Tempo': 143.087,
 'Time Signature': 4,
 'Valence': 0.351,
 'Album': 'Ella Swings Lightly',
 'Image': 'https://i.scdn.co/image/7f33383709040fba7b408770758b9b0d7ba4239a',
 'Explicit': False,
 'Popularity': 13,
 'Preview': None}

In [7]:
GRAMMY_WINNERS = os.path.join('..', 'data', 'grammy-winners.csv')
GRAMMY_WINNERS_ENRICHED = os.path.join('..', 'data', 'grammy-winners-enriched.csv')
SONGS_RELEASED = os.path.join('..', 'data', 'songs-released.csv')
SONGS_RELEASED_ENRICHED = os.path.join('..', 'data', 'songs-released-enriched.csv')
TOP_100_SONGS = os.path.join('..', 'data', 'top-100-songs.csv')
TOP_100_SONGS_ENRICHED = os.path.join('..', 'data', 'top-100-songs-enriched.csv')

In [8]:
grammy_winners_df = pd.read_csv(GRAMMY_WINNERS)
grammy_winners_df.rename(columns={'annualGrammy':'Year', 'awardFor':'Title', 'name':'Artist'}, inplace=True)
grammy_winners_df.drop(labels=['awardType', 'category'], axis=1, inplace=True)
grammy_winners_df['Year'] = grammy_winners_df.Year + 1957
grammy_winners_df.drop_duplicates(inplace=True)
grammy_winners_df = grammy_winners_df[['Year', 'Artist', 'Title']]
grammy_winners_df.head()

,Year,Artist,Title
0,1958,Domenico Modugno,Nel Blu Dipinto Di Blu (Volare)
1,1958,Henry Mancini,The Music From Peter Gunn
2,1958,"Domenico Modugno, songwriter (Domenico Modugno)",Nel Blu Dipinto Di Blu (Volare)
3,1958,Ella Fitzgerald,Ella Fitzgerald Sings The Irving Berlin Song Book
4,1958,Perry Como,Catch A Falling Star


In [9]:
songs_released_df = pd.read_csv(SONGS_RELEASED)
songs_released_df.drop_duplicates(inplace=True)
songs_released_df = songs_released_df[['Year', 'Artist', 'Title']]
songs_released_df.head()

,Year,Artist,Title
0,1958,Clyde McPhatter,A Lover's Question
1,1958,Pat Boone,A Wonderful Time Up There
2,1958,The Everly Brothers,All I Have to Do Is Dream
3,1958,Andy Williams,Are You Sincere
4,1958,The Playmates,Beep Beep


In [10]:
top_100_songs_df = pd.read_csv(TOP_100_SONGS)
top_100_songs_df.drop_duplicates(inplace=True)
top_100_songs_df = top_100_songs_df[['Year', 'Artist', 'Title']]
top_100_songs_df.drop_duplicates(inplace=True)
top_100_songs_df.head()

,Year,Artist,Title
0,1958,Domenico Modugno,Volare
1,1958,Everly Brothers,All I Have To Do Is Dream / Claudette
2,1958,Elvis Presley,Don't / I Beg of You
3,1958,David Seville,Witch Doctor
4,1958,Pérez Prado,Patricia


In [11]:
import time
def enrich_with_features(df):
    pb = IntProgress(min=0, max=len(df))
    display(pb)
    failures = []
    for index, row in df.iterrows():
        time.sleep(4)
        features = get_features(row['Title'], row['Artist'])
        pb.value += 1
        if features is None:
            failures.append((row['Title'], row['Artist']))
            continue
        for k, v in features.items():
            df.loc[index, k] = v
            
    print(f'Number of failures - {len(failures)}')
    print(failures)
    return df

In [12]:
grammy_winners_df.shape

(3911, 3)

In [13]:
grammy_winners_enriched_df = enrich_with_features(grammy_winners_df)
grammy_winners_enriched_df.head()

IntProgress(value=0, max=3911)

Number of failures - 781
[('Ella Fitzgerald Sings The Irving Berlin Song Book', 'Ella Fitzgerald'), ("Billy May's Big Fat Brass", 'Billy May'), ('Duets With A Spanish Guitar', 'Laurindo Almeida, guitar; Salli Terri, vocals'), ('Cross Country Suite', 'Nelson Riddle'), ('The Best Of The Stan Freberg Shows', 'Stan Freberg'), ('Operatic Recital', 'Renata Tebaldi, soprano'), ('Ella Swings Lightly', 'Ella Fitzgerald'), ('Beethoven: Sonatas No. 21 In C (Waldstein) And No. 18 In E Flat', 'Artur Rubinstein, piano'), ('Bjoerling In Opera', 'Jussi Bjoerling, tenor'), ('Inside Shelley Berman', 'Shelley Berman'), ('Victory At Sea, Vol. I', 'Robert Russell Bennett, conductor'), ("Alvin's Harmonica", 'David Seville'), ('Belafonte At Carnegie Hall', 'Harry Belafonte'), ('Dance With Basie', nan), ('Bartók: Music For Strings, Percussion And Celeste', 'Fritz Reiner, conductor'), ('Conversations With The Guitar', 'Laurindo Almeida, guitar'), ('The Spanish Guitars Of Laurindo Almeida', 'Laurindo Almeida, g

,Year,Artist,Title,URL,TrackId,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,...,Mode,Speechiness,Tempo,Time Signature,Valence,Album,Image,Explicit,Popularity,Preview
0,1958,Domenico Modugno,Nel Blu Dipinto Di Blu (Volare),https://open.spotify.com/track/006Ndmw2hHxvnLb...,006Ndmw2hHxvnLbJsBFnPx,0.987,0.518,216373.0,0.060,0.000008,...,1.0,0.0441,127.870,4.0,0.336,Tutto Modugno (Mister Volare),https://i.scdn.co/image/5e8c49f7a8d161c1d65109...,False,34.0,https://p.scdn.co/mp3-preview/a0e2ef35a1613d86...
1,1958,Henry Mancini,The Music From Peter Gunn,https://open.spotify.com/track/3BdPP6Xce6FUcfa...,3BdPP6Xce6FUcfaCFsnZIg,0.893,0.550,177733.0,0.318,0.881000,...,1.0,0.0313,138.037,4.0,0.620,The music from Peter Gunn,https://i.scdn.co/image/1ad2e8ce1f988c27678298...,False,16.0,https://p.scdn.co/mp3-preview/8380c4830866fc12...
2,1958,"Domenico Modugno, songwriter (Domenico Modugno)",Nel Blu Dipinto Di Blu (Volare),https://open.spotify.com/track/006Ndmw2hHxvnLb...,006Ndmw2hHxvnLbJsBFnPx,0.987,0.518,216373.0,0.060,0.000008,...,1.0,0.0441,127.870,4.0,0.336,Tutto Modugno (Mister Volare),https://i.scdn.co/image/5e8c49f7a8d161c1d65109...,False,34.0,https://p.scdn.co/mp3-preview/a0e2ef35a1613d86...
3,1958,Ella Fitzgerald,Ella Fitzgerald Sings The Irving Berlin Song Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1958,Perry Como,Catch A Falling Star,https://open.spotify.com/track/38YMdelhj62vJ6d...,38YMdelhj62vJ6d5a0wxMQ,0.864,0.628,148493.0,0.172,0.000000,...,1.0,0.0600,118.895,3.0,0.475,It's Impossible,https://i.scdn.co/image/0a33dd0dbd1b416c245c8c...,False,44.0,https://p.scdn.co/mp3-preview/9e2cfff3559c2e26...


In [14]:
grammy_winners_enriched_df.shape

(3911, 23)

In [15]:
grammy_winners_enriched_df.to_csv(GRAMMY_WINNERS_ENRICHED, index=False)

In [16]:
songs_released_df.shape

(6074, 3)

In [17]:
songs_released_enriched_df = enrich_with_features(songs_released_df)
songs_released_enriched_df.head()

IntProgress(value=0, max=6074)

retrying ...1secs
Number of failures - 239
[("Wouldn't It Be Fun!", 'Cole Porter'), ('The Same Old Army', 'The Swinging Corporals'), ("I'm a Honky Tonk Girl", 'Loretta Lynn'), ('Ya Mujabid Fe Sabil Allah', 'Umm Kulthum'), ("Housewife's Choice", 'Derrick & Patsy'), ("How Come There's No Dog Day?", 'Tommy Cooper'), ("I've Lost My Mummy", 'Rolf Harris'), ('Little Soldier Say Goodnight', 'Dave Sampson And The Hunters'), ("Pepy's Diary", 'Benny Hill'), ("That's What Girls Are Made For", 'The Spinners'), ('The Pretty Little Filly With The Ponytail', 'Sheldon Allman'), ("Don't Play That Song (You Lied)", 'Ben E. King'), ('Quietly Bonkers', 'Bernard Cribbins'), ("That's Old Fashioned (That's The Way Love Should Be)", 'The Everly Brothers'), ("(Let's Dance) The Screw", 'The Crystals'), ('All I Want For Christmas Is A Beatle', 'Dora Bryan'), ("Don't Say Nothin' Bad (About My Baby)", 'The Cookies'), ("Judy's Turn To Cry", 'Lesley Gore'), ('A Miss From Diss', 'The Singing Postman'), ('Bin Born A L

,Year,Artist,Title,URL,TrackId,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,...,Mode,Speechiness,Tempo,Time Signature,Valence,Album,Image,Explicit,Popularity,Preview
0,1958,Clyde McPhatter,A Lover's Question,https://open.spotify.com/track/6HBgCYResmsdmOu...,6HBgCYResmsdmOufyHVoNB,0.887,0.670,150267.0,0.436,0.000017,...,1.0,0.0554,83.023,4.0,0.951,Clyde,https://i.scdn.co/image/5f15ac3a4737e257e96c26...,False,42.0,https://p.scdn.co/mp3-preview/c9bc0f8e1f849364...
1,1958,Pat Boone,A Wonderful Time Up There,https://open.spotify.com/track/2ddcdLTP0DHU86V...,2ddcdLTP0DHU86VYETlEwm,0.757,0.790,127427.0,0.441,0.000056,...,1.0,0.0499,95.903,4.0,0.945,Pat Boone's Greatest Hits (Reissue),https://i.scdn.co/image/c3c266be0f8eef047d3fb5...,False,25.0,None
2,1958,The Everly Brothers,All I Have to Do Is Dream,https://open.spotify.com/track/7dblNGnRXEBrVJu...,7dblNGnRXEBrVJunazs2U5,0.839,0.534,143333.0,0.462,0.000000,...,1.0,0.0266,96.195,4.0,0.534,The Very Best of The Everly Brothers,https://i.scdn.co/image/fb7459c82a4c70cd707621...,False,64.0,https://p.scdn.co/mp3-preview/94d2e6419c9d6675...
3,1958,Andy Williams,Are You Sincere,https://open.spotify.com/track/6SW50CFLBpAWGDB...,6SW50CFLBpAWGDB0VtbTuW,0.868,0.488,168573.0,0.121,0.000026,...,1.0,0.0286,104.434,4.0,0.367,Best Of Andy Williams,https://i.scdn.co/image/fb3ed68b30105fbf0dd0f2...,False,14.0,https://p.scdn.co/mp3-preview/95675e5d97465bb3...
4,1958,The Playmates,Beep Beep,https://open.spotify.com/track/4uwounqJOKHVt0Y...,4uwounqJOKHVt0Y2Hu3dGE,0.719,0.519,150213.0,0.502,0.000000,...,0.0,0.2740,76.545,4.0,0.759,Right Said Fred: A Barrel Of Laughs,https://i.scdn.co/image/93a777dfffbe54b67105b3...,False,30.0,https://p.scdn.co/mp3-preview/0defe1566f6168a3...


In [18]:
songs_released_enriched_df.shape

(6074, 23)

In [19]:
songs_released_enriched_df.to_csv(SONGS_RELEASED_ENRICHED, index=False)

In [20]:
top_100_songs_df.shape

(5978, 3)

In [21]:
top_100_songs_enriched_df = enrich_with_features(top_100_songs_df)
top_100_songs_enriched_df.head()

IntProgress(value=0, max=5978)

retrying ...1secs
retrying ...1secs
retrying ...1secs
Number of failures - 193
[('All I Have To Do Is Dream / Claudette', 'Everly Brothers'), ("Don't / I Beg of You", 'Elvis Presley'), ('Sail Along Silvery Moon / Raunchy', 'Billy Vaughn'), ('Bird Dog / Devoted to You', 'Everly Brothers'), ("Stood Up / Waitin' In School", 'Ricky Nelson'), ("Wear My Ring Around Your Neck / Doncha' Think It's Time", 'Elvis Presley'), ("A Wonderful Time Up There / It's Too Soon to Know", 'Pat Boone'), ('Topsy Part 2 / Topsy Part 1', 'Cozy Cole'), ('My True Love / Leroy', 'Jack Scott'), ("Hard Headed Woman / Don't Ask Me Why", 'Elvis Presley'), ("Alvin's Harmonica", 'David Seville and The Chipmunks'), ("Alvin's Harmonica", 'David Seville & The Chipmunks'), ("What'd I Say, Pts. I & 2", 'Ray Charles'), ("I've Had It", 'The Bell Notes'), ("He'll Have to Stay", 'Jeanne Black'), ("Let's Think About Livin'", 'Bob Luman'), ("Don't Bet Money Honey", 'Linda Scott'), ("There's a Moon Out Tonight", 'The Capris'), ("It

,Year,Artist,Title,URL,TrackId,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,...,Mode,Speechiness,Tempo,Time Signature,Valence,Album,Image,Explicit,Popularity,Preview
0,1958,Domenico Modugno,Volare,https://open.spotify.com/track/4TVFmPiDtVtFe5P...,4TVFmPiDtVtFe5P9y5OwJ9,0.9630,0.297,274867.0,0.208,0.000022,...,0.0,0.0446,97.044,4.0,0.341,The 50 Greatest Vocal Hits,https://i.scdn.co/image/f769e1ec2c19cd07c9d333...,False,37.0,NaN
1,1958,Everly Brothers,All I Have To Do Is Dream / Claudette,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1958,Elvis Presley,Don't / I Beg of You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1958,David Seville,Witch Doctor,https://open.spotify.com/track/48skOMMuEQSfwo0...,48skOMMuEQSfwo0JZpfqjk,0.4550,0.600,142493.0,0.742,0.000000,...,1.0,0.0604,138.227,4.0,0.840,Alvin And The Chipmunks (Original Motion Pictu...,https://i.scdn.co/image/698c60bd6fd10a55dfdd76...,False,25.0,NaN
4,1958,Pérez Prado,Patricia,https://open.spotify.com/track/205EtD8tQnHDXMk...,205EtD8tQnHDXMkmtYiaHF,0.0862,0.704,140867.0,0.638,0.251000,...,1.0,0.0457,137.396,4.0,0.837,The Best Of Perez Prado: The Original Mambo #5,https://i.scdn.co/image/7700a28d06ef4e7ba51e2b...,False,37.0,https://p.scdn.co/mp3-preview/5173953cf5395923...


In [22]:
top_100_songs_enriched_df.shape

(5978, 23)

In [23]:
top_100_songs_enriched_df.to_csv(TOP_100_SONGS_ENRICHED, index=False)